In [1]:
# Our imports
import os

# Clustering of TIM
## What is the goal of this notebook? 

This notebook is intended to identify all unique chains in a dataset and their corresponding entity names. The results of this notebook can aid in the entity naming process by clustering all polymeric sequences in the dataset and retrieving the entity names associated with each member in each cluster.

NOTE:
>For clustering, we use **MMseqs2 coverage mode 1 (target mode)**.
>📖 For more details, see the [MMseqs2 documentation](https://github.com/soedinglab/MMseqs2).

Below is the workflow that this notebook follows. 

## Workflow

This notebook follows a step-by-step process to cluster PDB entries based on their sequences and extract meaningful information from the results.

1. __Setup__
- The notebook begins by creating the necessary directories to store all files generated throughout the workflow.

2. __Filtering SEQRES Entries__
- Using the provided PDB IDs file, we match each ID to its corresponding entry in `seqres.txt`. We then generate a new FASTA file that contains only the SEQRES entries corresponding to the selected PDB IDs.

3. __Clustering with MMseqs2__
- The filtered FASTA file is clustered using __MMseqs2__. The output is a `.tsv` file containing information about the clusters, such as the representatives and members.

4. __Generating Cluster FASTA Files__
- Using the `.tsv` file, we generate individual FASTA files for each cluster. Each file contains the SEQRES entries for all members of a given cluster, including the representative sequence.

5. __Extracting Entity Names__
- For each cluster FASTA file, we extract the entity names. These names are compiled into a singular `.txt` file, along with the count of occurrences.
    - Example entry of this file:
>6BVE_B_cluster.txt{'TRIOSEPHOSPHATE ISOMERASE': 26}{'TRIOSEPHOSPHATE ISOMERASE': 26, 'PROTEIN (TRIOSEPHOSPHATE ISOMERASE)': 4}
    - __6BVE_B_cluster.txt__ : The cluster representative
    - __{'TRIOSEPHOSPHATE ISOMERASE': 26}__ : The most common present entity in the cluster
    - __{'TRIOSEPHOSPHATE ISOMERASE': 26, 'PROTEIN (TRIOSEPHOSPHATE ISOMERASE)': 4}__ : All unique entities in the cluster

## Essential Files
This notebook uses two essential files to run:
1. `pdblist.txt`
2. `seqres.txt`

### PDBLIST File
This is a txt file containing a list of  PDB IDs, with one ID per line.

__Example:__
- PDB_ID_1
- PDB_ID_2
- PDB_ID_3

### SEQRES File
`seqres.txt` is a FASTA-formatted file containing sequences for all entries in the PDB. This file is generated by running Step 1 in the tutorial notebooks for PDBCleanV2.

__Example entry in seqres:__
>102m_A mol:protein length:154  MYOGLOBIN
MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDRFKHLKTEAEMKASEDLKKAGVTVLTALGAILKKKGHHEAELKPLAQSHATKHKIPIKYLEFISEAIIHVLHSRHPGNFGADAQGAMNKALELFRKDIAAKYKELGYQG

## Specify working directories
1. Let's make a directory where we will store everything we make in this notebook. As well as specifying our working directory

In [2]:
PROJDIR = './TIM/'
pdb_ids_path = PROJDIR + 'raw_bank/'
output_directory = PROJDIR + 'clustering_storage/'

In [3]:
# Making the directory where we will store everything
os.makedirs(output_directory, exist_ok=True)

* Lets save the file paths for both of our essential files

In [4]:
pdbid_file = PROJDIR + "pdblist.txt"
seqres_file = PROJDIR + "seqres.txt"

2. We'll also create a directory within our primary directory to store files related to the clustering mode for MMSeqs2 that we'll work with.
   - Mode 1

In [5]:
mode_directory = 'mode_1/'

In [6]:
mode_1_path = f'{output_directory}{mode_directory}'

In [7]:
os.makedirs(mode_1_path, exist_ok=True)

3. We will then create a directory within our mode directory, which will store FASTA files we will generate from the outputted clusters.

In [8]:
cluster_fasta_storage = 'cluster_files/'

In [9]:
cluster_fasta_storage_path = f'{mode_1_path}/{cluster_fasta_storage}'

In [10]:
os.makedirs(cluster_fasta_storage_path, exist_ok=True)

## How to create the 'pdblist.txt'

We will run the following two commands on the `raw_bank` directory to obtain the PDB IDs we are interested in.

1. `! ls path_to_directory | grep cif > pdbid_list_temp.txt`
2. `! sed 's/\.cif//g' pdbid_list_temp.txt > pdblist.txt`

__Explanation:__
- Line 1:
    - This command lists all files in the specified directory whose names contain the string `'cif'`, and saves the list to 'pdbid_list_temp.txt'.

- Line 2:
    - This command __removes the `.cif` extension__ from each line in `pdbid_list_temp.txt` and writes the cleaned results (just the base PDB IDs) to `pdblist.txt`.

In [11]:
! ls {pdb_ids_path} | grep cif > pdbid_list_temp.txt

In [12]:
! sed 's/\.cif//g' pdbid_list_temp.txt > pdblist.txt

In [13]:
! rm pdbid_list_temp.txt

In [14]:
! mv pdblist.txt {PROJDIR}

## Editing SEQRES
The entity names in the SEQRES file contain inconsistencies, including the use of special characters such as single quotation marks and asterisks. These irregularities can impact our clustering results. To address this, we will standardize the SEQRES entries by removing such inconsistencies.

In [15]:
output_seqres_path = PROJDIR + 'seqres2.txt'

In [16]:
with open(seqres_file, 'r') as input_file, open(output_seqres_path, 'w') as output_file:
    for meta_line, sequence_line in zip(input_file, input_file):
        # Clean up the line and split into parts
        attributes = meta_line.strip().split()
        
        # All attributes before the entity name
        other_attributes = attributes[:3]
        other_attributes = [attribute.upper() for attribute in other_attributes]
        
        # Rest is the entity name
        entity_name = ' '.join(attributes[3:])
        
        # Clean the entity name
        entity_name_updated = entity_name.replace("'", "").replace("*", "-")
        
        # Reconstructing the meta_line
        new_meta_line = ' '.join(other_attributes + [entity_name_updated]) + '\n'
        
        # Write to output
        output_file.write(new_meta_line)
        output_file.write(sequence_line)

In [17]:
# Replaces our old seqres file
! mv {PROJDIR}/seqres2.txt {PROJDIR}/seqres.txt

## PDB IDs File to FASTA
We start with our **PDB IDs** file, and match the PDB IDs in the file to their corresponding entries in the `seqres` file.  

### Define functions
We’ll define a few __helper functions__ essential for performing this task:

In [18]:
def get_attributes_seqres(file_path, attribute='PDBID_CHID'):
    """
    Extracts the specified attributes from the metadata line of a FASTA file.

    file_path(str) : File path leading to seqres.txt
    attribute(str) : Metadata attribute to extract. Options are 'ENTITY_LABEL',
                    'PDBID', 'CHID', 'PDBID_CHID'. The default is PDBID_CHID.
    Returns : 
        attribute_list(list) : List of extracted attributes.
    """
    attribute_list = []
    attribute = attribute.upper()

    # Had to add the encoding parameter due to an error that kept occuring when reading the file.
    with open(file_path, 'r', encoding='latin-1') as seqres:
        for i, line in enumerate(seqres):
            
            if i % 2 == 0:
                
                if attribute == 'ENTITY_LABEL':
                    entity_name = ' '.join(line.split()[3:])
                    attribute_list.append(entity_name.upper())
                # Every other attribute outside of entity label is found within the first item 
                # obtained after we perform the split function on the metadataline,
                # Therefore we perform the following code after the 'else' function.
                
                else:
                    line = line.split()[0][1:]
                # Everything before the '_' is the PDB ID]
                    if attribute == 'PDBID':
                        # Added .upper()
                        attribute_list.append(line.split('_')[0].upper())
                # Everything after the '_' is the chain ID
                    elif attribute == 'CHID':
                        attribute_list.append(line.split('_')[1])
                    elif attribute == 'PDBID_CHID':
                        pdb_id = line.split('_')[0].upper()
                        chid = line.split('_')[1]
                        
                        pdb_chid = pdb_id + '_' + chid
                        attribute_list.append(pdb_chid)
            
            elif i % 2 == 1 and attribute == 'SEQUENCE':
                attribute_list.append(line.strip().upper())
                    
    return attribute_list

In [19]:
def seqres_entry_dict(file_path):
    """
    Returns a dictionary with the keys being the PDB entry ID and chain ID (e.g. pdbID_chID) and the values
    as [meta data, sequence].

    file_path : file path leading to seqres

    Note : The chain ID is case sensitive, below we make sure that the 
    """
    entry_dict = {}
    with open(file_path, 'r') as file:
        for meta_line, sequence_line in zip(file, file):
            
            # Here we capitalize everything in pdb_chid except for the chain id
            pdbchid_label_fasta = meta_line.split()[0][1:]
            
            pdb_id = pdbchid_label_fasta.split('_')[0].upper()
            ch_id = pdbchid_label_fasta.split('_')[1]
            
            pdb_chid = pdb_id + '_' + ch_id

            key = pdb_chid

            # We obtain all other components in our metaline (e.g mol, length, entity name)
            rest_line_cap = [word.upper() for word in meta_line.split()[1:]]
            
            # We construct our metaline with our capitalized components
            meta_line = '>' + pdb_chid + ' ' +' '.join(rest_line_cap) + '\n'

            # We make our dictionary from our capitalzed seqres components
            entry_dict[key] = [meta_line, sequence_line.upper()]

    return entry_dict 

- **Task**: Here we save the `PDB ID`'s from the provided txt file containing only PDB IDs

In [20]:
PDB_ID_list = []
with open (pdbid_file, 'r') as file:
    for line in file:
        PDB_ID_list.append(line[:4].upper())

test_pdbid = PDB_ID_list[0]
print(f'An attribute from our list: {PDB_ID_list[PDB_ID_list.index(test_pdbid)]}')

An attribute from our list: 1AG1


- **Task**: Here we extract the `pdbID_chID` labels from `seqres` and store them in a __list__

In [21]:
pdb_chid_list = get_attributes_seqres(seqres_file, 'PDBID_CHID')

test_pdbid2 = [pdb for pdb in pdb_chid_list if pdb[:4] == test_pdbid][0]
print(f'An attribute from our list: {pdb_chid_list[pdb_chid_list.index(test_pdbid2)]}')

An attribute from our list: 1AG1_O


Below we extract the matching `pdbID_chID` from `seqres` that corresponds to the `PDB ID` in the `pdb_id` file, and store it in a list.  

- **Example**:  
  - `PDB ID`: `1AG1`  
  - `SEQRES pdbID_chID`: `1AG1_A`  

Since the PDB IDs match, we store the `SEQRES pdbID_chID` (e.g., `1AG1_A`) in a list.  

In [22]:
matching_labels = [label for label in pdb_chid_list if label.split('_', 1)[0] in PDB_ID_list]
print(f'An attribute from our list: {matching_labels[matching_labels.index(test_pdbid2)]}')

An attribute from our list: 1AG1_O


- **Task**: Below we make a dictionary out of our `seqres` file. (Read function documentation for further clarity)

In [23]:
seqres_dict = seqres_entry_dict(seqres_file)
print(f'An attribute from our dictionary: {seqres_dict[test_pdbid2]}')

An attribute from our dictionary: ['>1AG1_O MOL:PROTEIN LENGTH:250 TRIOSEPHOSPHATE ISOMERASE\n', 'MSKPQPIAAANWKCNGSQQSLSELIDLFNSTSINHDVQCVVASTFVHLAMTKERLSHPKFVIAAQNAIAKSGAFTGEVSLPILKDFGVNWIVLGHSERRAYYGETNEIVADKVAAAVASGFMVIACIGETLQERESGRTAVVVLTQIAAIAKKLKKADWAKVVIAYEPVWAIGTGKVATPQQAQEAHALIRSWVSSKIGADVAGELRILYGGSVNGKNARTLYQQRDVNGFLVGGASLKPEFVDIIKATQ\n']


- **Task**: with all of our components, we now can write a FASTA file containing only the structures we are interested in.
- **Output**: A FASTA file within our mode_1 directory

In [24]:
# Specifying what I wish for my FASTA file to be called
fasta_output_name = 'tim_seqres.txt'

In [25]:
with open(f'{mode_1_path}/{fasta_output_name}', 'w') as file:
    for label in matching_labels:
        lines_to_write = seqres_dict[label]
        if lines_to_write:
            file.writelines(lines_to_write)

In [26]:
print("The first 5 entries of our 'tim_seqres.txt'\n")
! head {mode_1_path}/{fasta_output_name}

The first 5 entries of our 'tim_seqres.txt'

>1AG1_O MOL:PROTEIN LENGTH:250 TRIOSEPHOSPHATE ISOMERASE
MSKPQPIAAANWKCNGSQQSLSELIDLFNSTSINHDVQCVVASTFVHLAMTKERLSHPKFVIAAQNAIAKSGAFTGEVSLPILKDFGVNWIVLGHSERRAYYGETNEIVADKVAAAVASGFMVIACIGETLQERESGRTAVVVLTQIAAIAKKLKKADWAKVVIAYEPVWAIGTGKVATPQQAQEAHALIRSWVSSKIGADVAGELRILYGGSVNGKNARTLYQQRDVNGFLVGGASLKPEFVDIIKATQ
>1AG1_T MOL:PROTEIN LENGTH:250 TRIOSEPHOSPHATE ISOMERASE
MSKPQPIAAANWKCNGSQQSLSELIDLFNSTSINHDVQCVVASTFVHLAMTKERLSHPKFVIAAQNAIAKSGAFTGEVSLPILKDFGVNWIVLGHSERRAYYGETNEIVADKVAAAVASGFMVIACIGETLQERESGRTAVVVLTQIAAIAKKLKKADWAKVVIAYEPVWAIGTGKVATPQQAQEAHALIRSWVSSKIGADVAGELRILYGGSVNGKNARTLYQQRDVNGFLVGGASLKPEFVDIIKATQ
>1AW1_A MOL:PROTEIN LENGTH:256 TRIOSEPHOSPHATE ISOMERASE
MRHPVVMGNWKLNGSKEMVVDLLNGLNAELEGVTGVDVAVAPPALFVDLAERTLTEAGSAIILGAQNTDLNNSGAFTGDMSPAMLKEFGATHIIIGHSERREYHAESDEFVAKKFAFLKENGLTPVLCIGESDAQNEAGETMAVCARQLDAVINTQGVEALEGAIIAYEPIWAIGTGKAATAEDAQRIHAQIRAHIAEKSEAVAKNVVIQYGGSVKPENAAAYFAQPDIDGALVGGAALDAKSFAAIAKAAAEAKA
>1AW1_B MOL:PROTEIN LENGT

# Performing Clustering
Using the program MMSeqs2, we will perform the clustering of our created FASTA file.

## MMseqs2 clustering steps
I will run the following three lines:
1. __mmseqs createdb fasta_file.txt pdbDB__
   - Converts the input sequence file seqres.txt into an MMseqs2 database named pdbDB (pdbDB was a name of my choosing).

2. __mmseqs cluster pdbDB clusterDB tmp --min-seq-id 0.95 --cluster-mode 2 --cov-mode 1 -c 0.8__
   - Clusters the sequences in `pdbDB` based on a **minimum sequence identity** (`--min-seq-id`) and **coverage** (`-c`).  
   - The example values here are `0.95` for sequence identity and `0.8` for coverage, but you can **choose any values** suitable for your analysis. We also use `clustering mode 2` and `coverage mode 1`
   - Outputs the clustering result into `clusterDB`.

3. __mmseqs createtsv pdbDB pdbDB clusterDB clusters.tsv__
   - Outputs the clusters from clusterDB into a readable TSV file (clusters.tsv).

In [27]:
# For a good part of this notebook, we will be working within this directory
os.chdir(mode_1_path)

In [28]:
!mmseqs createdb {fasta_output_name} pdbDB 

createdb tim_seqres.txt pdbDB 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[607] 0s 5ms
Time for merging to pdbDB_h: 0h 0m 0s 3ms
Time for merging to pdbDB: 0h 0m 0s 2ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 22ms


In [29]:
!mmseqs cluster pdbDB clusterDB tmp --min-seq-id 0.95 --cluster-mode 2 --cov-mode 1 -c 0.8

Create directory tmp
cluster pdbDB clusterDB tmp --min-seq-id 0.95 --cluster-mode 2 --cov-mode 1 -c 0.8 

MMseqs Version:                     	14.7e284
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	1
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask residues probability     

align tmp/12435702776915990674/linclust/5764353067963804588/input_step_redundancy tmp/12435702776915990674/linclust/5764353067963804588/input_step_redundancy tmp/12435702776915990674/linclust/5764353067963804588/pref_rescore2 tmp/12435702776915990674/linclust/5764353067963804588/aln --sub-mat 'aa:blosum62.out,nucl:nucleotide.out' -a 0 --alignment-mode 3 --alignment-output-mode 0 --wrapped-scoring 0 -e 0.001 --min-seq-id 0.95 --min-aln-len 0 --seq-id-mode 0 --alt-ali 0 -c 0.8 --cov-mode 1 --max-seq-len 65535 --comp-bias-corr 0 --comp-bias-corr-scale 1 --max-rejected 2147483647 --max-accept 2147483647 --add-self-matches 0 --db-load-mode 0 --pca substitution:1.100,context:1.400 --pcb substitution:4.100,context:5.800 --score-bias 0 --realign 0 --realign-score-bias -0.2 --realign-max-seqs 2147483647 --corr-score-weight 0 --gap-open aa:11,nucl:5 --gap-extend aa:1,nucl:2 --zdrop 40 --threads 8 --compressed 0 -v 3 

Compute score, coverage and sequence identity
Query database size: 85 type: Am

mergeclusters pdbDB tmp/12435702776915990674/clu_redundancy tmp/12435702776915990674/linclust/5764353067963804588/pre_clust tmp/12435702776915990674/linclust/5764353067963804588/clust --threads 8 --compressed 0 -v 3 

Clustering step 1
[=================================================================] 100.00% 85 0s 1ms
Clustering step 2
[=================================================================] 100.00% 83 0s 2ms
Write merged clustering
[=================================================================] 100.00% 675 0s 6ms
Time for merging to clu_redundancy: 0h 0m 0s 0ms
Time for processing: 0h 0m 0s 10ms
createsubdb tmp/12435702776915990674/clu_redundancy pdbDB tmp/12435702776915990674/input_step_redundancy -v 3 --subdb-mode 1 

Time for merging to input_step_redundancy: 0h 0m 0s 0ms
Time for processing: 0h 0m 0s 4ms


prefilter tmp/12435702776915990674/input_step_redundancy tmp/12435702776915990674/input_step_redundancy tmp/12435702776915990674/pref_step0 --sub-mat 'aa:blosum62.out,nucl:nucleotide.out' --seed-sub-mat 'aa:VTML80.out,nucl:nucleotide.out' -s 1 -k 0 --k-score seq:2147483647,prof:2147483647 --alph-size aa:21,nucl:5 --max-seq-len 65535 --max-seqs 20 --split 0 --split-mode 2 --split-memory-limit 0 -c 0.8 --cov-mode 1 --comp-bias-corr 0 --comp-bias-corr-scale 1 --diag-score 0 --exact-kmer-matching 0 --mask 1 --mask-prob 0.9 --mask-lower-case 0 --min-ungapped-score 0 --add-self-matches 0 --spaced-kmer-mode 1 --db-load-mode 0 --pca substitution:1.100,context:1.400 --pcb substitution:4.100,context:5.800 --threads 8 --compressed 0 -v 3 

Query database size: 83 type: Aminoacid
Estimated memory consumption: 977M
Target database size: 83 type: Aminoacid
Index table k-mer threshold: 154 at k-mer size 6 
Index table: counting k-mers
[=================================================================

In [30]:
!mmseqs createtsv pdbDB pdbDB clusterDB clusters.tsv

createtsv pdbDB pdbDB clusterDB clusters.tsv 

MMseqs Version:                 	14.7e284
First sequence as representative	false
Target column                   	1
Add full header                 	false
Sequence source                 	0
Database output                 	false
Threads                         	8
Compressed                      	0
Verbosity                       	3

Time for merging to clusters.tsv: 0h 0m 0s 1ms
Time for processing: 0h 0m 0s 4ms


In [31]:
# Cleaning up files and directories we do not need
!rm pdbDB*
!rm -r tmp
!rm clusterDB*

## Interpreting Our Clustering
Next, we will generate **FASTA files** for each cluster and extract additional attributes.  

- Below, we define helper **functions** to perform these tasks efficiently.

In [32]:
# run the code block to define the function
def map_unique(list1, list2):
    """
    Creates a mapping from each unique element in list2 to a set of corresponding elements from list1.

    list1 : A list of values to be grouped.
    list2 : A list of keys which will be used for grouping.
    """
    zip_values = zip(list1, list2)
    values_map = {}
    
    if len(list1) != len(list2):
        return print('Both lists must be of equal lengths.')
    
    else:
        for value1, value2 in zip_values:
            if value2 not in values_map:
                values_map[value2] = set()
            values_map[value2].add(value1)
        
    return values_map

In [33]:
def cleaning_clusters(file_path):
    """
    Creates a dictionary with unique cluster representatives as keys and grouped members as values.
    
    file_path : path of .tsv file containing information on cluster representatives and members.
    """
    # fix indexing, extract whole thing
    with open(file_path, 'r') as file:
        label_list = []
        representatives = []
        members = []
        for i, line in enumerate(file):
            label_list.append(line)
        for label in label_list:
            representatives.append(label.split()[0])
            members.append(label.split()[1])
            
        cluster_dict = map_unique(members, representatives)
    return cluster_dict

In [34]:
def writing_cluster_FASTA(seqres_dict, cluster_dict, output_directory):
    """
    Created FASTA files out of the clusters made using MMSeqs2.

    seqres_dict: dictionary made out of our seqres file utalizing seqres_entry_dict function
    cluster_dict: Dictionary built off of the clusters.tsv having used the cleaning_clusters function
    output_directory: directory where outputted files will be stored
    """
    for key in cluster_dict:
        with open(f'{output_directory}/{key}_cluster.txt', 'w') as file:
            entries_list = list(cluster_dict[key])
            for entry in entries_list:
                lines_to_write = seqres_dict[entry]
                if lines_to_write:  
                    file.writelines(lines_to_write)
                else:
                    print(f'{entry} not in seqres.txt')

- **Task**: We create a dictionary out of our 'clusters.tsv' file. (read function documentation for more information)

In [35]:
cluster_dict = cleaning_clusters('clusters.tsv')

cluster_key = list(cluster_dict.keys())[0]
print(f'An attribute of our cluster dictionary:\n{cluster_key} : {cluster_dict[cluster_key]}')

An attribute of our cluster dictionary:
3GVG_A : {'3TAO_A', '3TA6_B', '3GVG_B', '3TAO_B', '3TA6_A', '3GVG_A'}


- Additional information on our created clusters:

In [36]:
member_count = []
for key in cluster_dict.keys():
    member_count.append(len(cluster_dict[key]))

print(f"cluster member counts : {sum(member_count)}")
print(f'cluster representatives count : {len(cluster_dict.keys())}')

cluster member counts : 675
cluster representatives count : 80


### Writing FASTA files out of our clusters

We now run a function to generate **FASTA files** from our clusters and save them in the `cluster_files` directory.

In [37]:
writing_cluster_FASTA(seqres_dict, cluster_dict, cluster_fasta_storage)

In [38]:
print(f'First 5 files in {cluster_fasta_storage[:cluster_fasta_storage.index('/')]}:')
!ls {cluster_fasta_storage}| head -n 5

First 5 files in cluster_files:
1AG1_O_cluster.txt
1AW1_A_cluster.txt
1B9B_A_cluster.txt
1HG3_G_cluster.txt
1I45_A_cluster.txt


### Writing our unique entity names file

In this step, we count every **unique entity** and identify the **most frequent entity** in our cluster FASTA files.  
We append this information, along with the **total number of entities** across all clusters, into a `.txt` file.
- Let's define functions needed for this task 

In [39]:
def obtaining_label_counts(file_path):
    """
    Returns a dictionary, with the keys being the unique entity labels in a FASTA file 
    and the values being the counts of those keys in the file.

    file_path : path to FASTA file
    """
    label_list = get_attributes_seqres(file_path, 'ENTITY_LABEL')
    unique_label_list = set(label_list)
    counts = []
    
    for unique_label in unique_label_list:
        count = 0
        for label in label_list:
            if label == unique_label:
                count += 1
        counts.append(count)
    return dict(zip(unique_label_list, counts))

In [40]:
def obtain_high_label(label_dict):
    """
    Takes dictionary from obtaining_label_counts and returns a dictionary containing the key and value of 
    the most present entity label in the FASTA file

    label_dict : dictionary from obtaining_label_counts()
    """
    if len(label_dict.keys()) > 1:
        label_dict = dict(sorted(label_dict.items(), key=lambda item: item[1], reverse=True))
        return {list(label_dict.keys())[0] : label_dict[list(label_dict.keys())[0]]}
    else:
        return {}

In [41]:
def writing_labels_file(clusters_directory_path, output_directory):
    """
    Creates a text file where each line contains the name of a cluster file, the most frequent 
    entity name for that cluster (along with their counts), and the unique entity names within 
    that cluster (also with its count).

    clusters_directory_path : path to directory containing the cluster FASTA files

    output_directory : directory where the newly created txt file will be stored
    """
    lines_to_write = []
    structure_count = 0

    #This chunk removes a file we dont need.
    ds_store_path = os.path.join(clusters_directory_path, '.DS_Store')
    if os.path.exists(ds_store_path):
        os.remove(ds_store_path)
        
    for file in os.listdir(clusters_directory_path):
            
        full_path = os.path.join(clusters_directory_path, file)

        #This line counts how main chains are in the current file its reading, 
        # and adds up that total to its count. It repeats that for every file in the cluster
        structure_count += len(get_attributes_seqres(full_path, 'chid'))
        test_dict = obtaining_label_counts(full_path)
        high_label = obtain_high_label(test_dict)
            
        if high_label:
            line = [f'{file}',f'{high_label}',f'{test_dict}']
        else:
            line = [f'{file}',f'{test_dict}']
            
        lines_to_write.append(line)
    
    with open(f'{output_directory}/unique_entity_names.txt', 'w') as curr_file:
        curr_file.writelines(f'TOTAL CHAINS : {structure_count}\n\n')
        for line in lines_to_write:
            curr_file.writelines(line)
            curr_file.writelines('\n')

- **Task:** creating our unique entity names file

In [42]:
writing_labels_file(cluster_fasta_storage, '.')

In [43]:
print('First 5 lines of our unique_entity_names.txt')
!head -n 5 unique_entity_names.txt

First 5 lines of our unique_entity_names.txt
TOTAL CHAINS : 675

1HG3_G_cluster.txt{'TRIOSEPHOSPHATE ISOMERASE': 8}
4Y90_A_cluster.txt{'TRIOSEPHOSPHATE ISOMERASE': 4}
1I45_A_cluster.txt{'TRIOSEPHOSPHATE ISOMERASE': 16}
